# Hello World Examples

In this notebook, we will walk you through some Hello World examples in `NVFlare/examples/hello-world` to get familiar with basic workflow of NVIDIA FLARE.

We will run the examples in FLARE with the POC mode using the [FLARE API](../tutorials/flare_api.ipynb). You can also run these examples in the [FLARE simulator](../tutorials/flare_simulator.ipynb).

Each example below is self-contained. You can start from any example, but you must run through the 3 steps of each example in sequence.

## Prerequisites
Before you can run the examples here, the following preparation work must be done:

1. Install a virturalenv following the instructions in [README.md](https://github.com/NVIDIA/NVFlare/tree/dev/examples)
2. Install Jupyter Lab and install a new kernel for the virtualenv called `nvflare_example`
3. Install NVFlare following this [notebook](../nvflare_setup.ipynb)
4. Start NVFlare in POC mode following this [notebook](../tutorials/setup_poc.ipynb). All the examples in this notebook require 2 clients to run.

Just to quickly recap the NVFLARE installation and POC installation steps:

**NVFLARE Installation**

```! pip install nvflare```

Check nvflare version with ``` nvflare --version ``` or  ```nvflare -V```

```! nvflare -V```



**Start NVFLARE in POC mode**

nvflare POC commands
```
   nvflare poc --prepare -n N
   nvflare poc --start -ex admin  
   nvflare poc --stop 
   nvflare poc --clean
```

``-ex admin`` means exclude FLARE Console (Admin Console) 

  the default workspace = /tmp/nvflare/poc


```python

import os
NVFLARE_HOME=os.path.abspath(os.path.join(os.getcwd(), "../.."))
%env NVFLARE_HOME={NVFLARE_HOME}

```

```! echo ${NVFLARE_HOME}```

```! echo y | nvflare poc --prepare -n 2 ```

```ls -al  /tmp/nvflare/poc/admin/transfer```

you cut paste the following line in a terminal to start FL system

_```nvflare poc --start -ex admin```_

you can use

_```hello_world_pre_start.sh```_

scripts to run previous steps and start POC **from a terminal**. 


**Note**

using the bash background from the cell such as 

```
%%bash -bg
nvflare poc --start -ex admin

```
not always reliable work!


## Check FL System Status

**Warning**:  this step will fail if FL system is not running


Now, we are ready to check FL system status



In [ ]:
import os
import time

from nvflare.fuel.flare_api.flare_api import new_insecure_session
from nvflare.fuel.flare_api.flare_api import NoConnection 

workspace = "/tmp/nvflare/poc"
admin_dir = os.path.join(workspace, "admin")

# the following try/except is usually not needed, we need it here to handle the case when you "Run all cells" or use notebook automation. 
# in the "Run all cells" case, JupyterLab seems to try to connect to the server before it starts (even though the execution is supposed to be sequential),
# which will result in a connection timeout. We use try/except to capture the scenario since extra sleep time doesn't seem to help.

try: 
   sess = new_insecure_session(admin_dir, timeout=5)
except NoConnection:
    time.sleep(10)
    
    
flare_not_ready = True
while flare_not_ready: 
    print("trying to connect to server")
    sess = new_insecure_session(admin_dir)
    sys_info = sess.get_system_info()

    print(f"Server info:\n{sys_info.server_info}")
    print("\nClient info")
    for client in sys_info.client_info:
        print(client)
    flare_not_ready = len( sys_info.client_info) < 2
        
    time.sleep(2)


### Utilities

**Monitoring Job**

You can choose your monitoring output, here is one function to display the job information 

In [ ]:
import json
from nvflare.fuel.flare_api.flare_api import Session

def status_monitor_cb(
        session: Session, job_id: str, job_meta, *cb_args, **cb_kwargs
    ) -> bool:
    if job_meta["status"] == "RUNNING":
        if cb_kwargs["cb_run_counter"]["count"] < 3 or cb_kwargs["cb_run_counter"]["count"]%15 == 0:
            print(job_meta)            
        else: 
            print(".", end="")
    else:
        print("\n" + str(job_meta))
    
    cb_kwargs["cb_run_counter"]["count"] += 1
    return True



def format_json( data: dict): 
    print(json.dumps(data, sort_keys=True, indent=4,separators=(',', ': ')))


## Hello Scatter and Gather

The example job in `hello-world/hello-numpy-sag/jobs/hello-numpy-sag` demonstrate the scatter and gather workflow. See [this](https://nvflare.readthedocs.io/en/main/examples/hello_scatter_and_gather.html#hello-scatter-and-gather) for the details of the example.

### 1. Submit job using FLARE API

Starting a FLARE API session and submit the `hello-numpy-sag` job

In [ ]:
import os
from nvflare.fuel.flare_api.flare_api import new_insecure_session

poc_workspace = "/tmp/nvflare/poc"
admin_dir = os.path.join(poc_workspace, "admin")
sess = new_insecure_session(admin_dir)

job_folder = os.path.join(os.getcwd(), "hello-numpy-sag/jobs/hello-numpy-sag")
job_id = sess.submit_job(job_folder)

print(f"Job is running with ID {job_id}")

### 2. Wait for the job

The command `monitor_job()` will wait for the job till it's done.

In [ ]:
list_jobs_output_detailed = sess.list_jobs(detailed=True)
print(format_json(list_jobs_output_detailed))

In [ ]:
sess.get_job_meta(job_id)

In [ ]:
sess.monitor_job(job_id, cb=status_monitor_cb, cb_run_counter={"count":0})

### 3. Get the result


In [ ]:
import numpy as np
result = sess.download_job_result(job_id)
array = np.load(result + "/workspace/models/server.npy")
print(array)

#### Clean up result directory

In [ ]:
rm -r {result}

## Hello Cross-Site Validation

The example job in `hello-world/hello-numpy-cross/jobs/hello-numpy-cross` demonstrates how to perform cross site validation after training.

Please refer to the [documentation](https://nvflare.readthedocs.io/en/main/examples/hello_cross_val.html) for the details.

### 1. Submit job using FLARE API

Starting a FLARE API session and submit the `hello-numpy-cross-val` job

In [ ]:
import os
from nvflare.fuel.flare_api.flare_api import new_insecure_session

poc_workspace = "/tmp/nvflare/poc"
admin_dir = os.path.join(poc_workspace, "admin")
sess = new_insecure_session(admin_dir)

job_folder = os.path.join(os.getcwd(), "hello-numpy-cross-val/jobs/hello-numpy-cross-val")
job_id = sess.submit_job(job_folder)

print(f"Job is running with ID {job_id}")

### 2. Wait for the job

In [ ]:
sess.get_job_meta(job_id)

In [ ]:
sess.monitor_job(job_id, cb=status_monitor_cb, cb_run_counter={"count":0})

### 3. Get the result

In [ ]:
import json
import pprint

result = sess.download_job_result(job_id)
with open(result + "/workspace/cross_site_val/cross_val_results.json", "r") as f:
  cross_val_result = json.load(f)

pp = pprint.PrettyPrinter(indent=2)
pp.pprint(cross_val_result)

#### Clean up result directory

In [ ]:
rm -r {result}

## Hello Cyclic Weight Transfer

This example uses the CyclicController workflow to implement [Cyclic Weight Transfer](https://pubmed.ncbi.nlm.nih.gov/29617797/) with TensorFlow as the deep learning training framework. The job is `hello-world/hello-cyclic/jobs/hello-cyclic`.

To use this example, tensorflow must be installed using the `requirements.txt`,

    pip install -r hello-world/hello-cyclic/requirements.txt
    
This examples needs access to [MNIST dataset](http://yann.lecun.com/exdb/mnist/)


In [ ]:
! pwd

In [ ]:
! pip install -r ../hello-world/hello-cyclic/requirements.txt    


### 1. Submit job using FLARE API

Starting a FLARE API session and submit the hello-cyclic job

In [ ]:
import os
from nvflare.fuel.flare_api.flare_api import new_insecure_session

poc_workspace = "/tmp/nvflare/poc"
admin_dir = os.path.join(poc_workspace, "admin")
sess = new_insecure_session(admin_dir)

job_folder = os.path.join(os.getcwd(), "hello-cyclic/jobs/hello-cyclic")
job_id = sess.submit_job(job_folder)
print(f"Job is running with ID {job_id}")

### 2. Wait for the job

In [ ]:
sess.monitor_job(job_id)

### 3. Get the result

In [ ]:
from nvflare.fuel.utils import fobs
from nvflare.app_common.decomposers import common_decomposers
import pprint

# This example stores numpy arrays in FOBS format. Decomposers for Numpy is not registered automatically.
common_decomposers.register()

result = sess.download_job_result(job_id)
with open(result + "/workspace/app_server/tf2weights.fobs", "rb") as f:
    bytes = f.read()

weights = fobs.loads(bytes)

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(weights)

#### Clean up result directory

In [ ]:
rm -r {result}

## Hello PyTorch

This example demonstrates how to use NVFlare with the popular deep learning framework PyTorch. The job is `hello-world/hello-pt/jobs/hello-pt`.

Refer to the [documentation](https://nvflare.readthedocs.io/en/main/examples/hello_pt.html) for details.

To use this example, PyTorch must be installed using the `requirements.txt`,

    pip install -r hello-world/hello-pt/requirements.txt
    
This examples also needs access to CIFAR10 dataset.


In [ ]:
! pip install -r ../hello-world/hello-pt/requirements.txt    

### 1. Submit job using FLARE API

Starting a FLARE API session and submit the hello-pt job

In [ ]:
import os
from nvflare.fuel.flare_api.flare_api import new_insecure_session

poc_workspace = "/tmp/nvflare/poc"
admin_dir = os.path.join(poc_workspace, "admin")
sess = new_insecure_session(admin_dir)

job_folder = os.path.join(os.getcwd(), "hello-pt/jobs/hello-pt")
job_id = sess.submit_job(job_folder)

print(f"Job is running with ID {job_id}")

### 2. Wait for the job

In [ ]:
sess.monitor_job(job_id)

### 3. Get the result

In [ ]:
import os
import pprint
import torch

print("this will take a bit of time")
result = sess.download_job_result(job_id)
model_path = os.path.join(result, "workspace/app_server/FL_global_model.pt")

model = torch.load(model_path)

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(model)

#### Clean up result directory

In [ ]:
rm -r {result}

## Hello TensorFlow 2

This example demonstrates how to use NVFlare with the popular deep learning framework TensorFlow 2. The job is `examples/hello-world/hello-tf2/jobs/hello-tf2`.

Refer to the [documentation](https://nvflare.readthedocs.io/en/main/examples/hello_tf2.html) for details.

To use this example, PyTorch must be installed using the `requirements.txt`,

    pip install -r hello-world/hello-tf2/requirements.txt
    
This examples also needs access to [MNIST dataset](http://yann.lecun.com/exdb/mnist/)

In [ ]:
! pip install -r hello-tf2/requirements.txt

### 1. Submit job using FLARE API

Starting a FLARE API session and submit the hello-tf2 job

This time, we tail the server log

In [ ]:
import os
from nvflare.fuel.flare_api.flare_api import new_insecure_session

poc_workspace = "/tmp/nvflare/poc"
admin_dir = os.path.join(poc_workspace, "admin")
sess = new_insecure_session(admin_dir)

job_folder = os.path.join(os.getcwd(), "hello-tf2/jobs/hello-tf2")
job_id = sess.submit_job(job_folder)                          
print(f"Job is running with ID {job_id}")

In [ ]:
! tail -100 /tmp/nvflare/poc/server/log.txt

In [ ]:
import json

list_jobs_output = sess.list_jobs()
print(format_json(list_jobs_output))


### 2. Wait the job

In [ ]:
sess.monitor_job(job_id)

### 3. Get the result

In [ ]:
from nvflare.fuel.utils import fobs
from nvflare.app_common.decomposers import common_decomposers
import pprint

common_decomposers.register()
result = sess.download_job_result(job_id)
with open(result + "/workspace/app_server/tf2weights.fobs", "rb") as f:
    bytes = f.read()

weights = fobs.loads(bytes)

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(weights)

#### Clean up result directory

In [ ]:
rm -r {result}

## Cleanup
We need to shutdown NVFLARE system and clean up POC workspace. This can be down in the following steps. 
you can change the cell into the code cell from markdown cell.
or you can simply excute

```hw_post_cleanup.sh```

bash scripts which does the followings


```
! nvflare poc --stop
! sleep 3
```

``` !ps -eaf | grep nvflare ```



```! nvflare poc --clean```
